In [6]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import time
import pickle

# Only from 1983–84 onward (nba_api has data)
seasons = [f"{year}-{str(year + 1)[-2:]}" for year in range(1983, 2024)]

regular_ids = []
playoff_ids = []

for season in seasons:
    print(f"\nSeason: {season}")
    
    # === Regular Season ===
    time.sleep(1)
    reg = leaguegamefinder.LeagueGameFinder(
        season_nullable=season,
        season_type_nullable="Regular Season"
    )
    reg_df = reg.get_data_frames()[0]
    regular_ids.extend(reg_df["GAME_ID"].unique().tolist())
    print(f"Regular games: {len(reg_df)}")

    # === Playoffs ===
    time.sleep(1)
    po = leaguegamefinder.LeagueGameFinder(
        season_nullable=season,
        season_type_nullable="Playoffs"
    )
    po_df = po.get_data_frames()[0]
    playoff_ids.extend(po_df["GAME_ID"].unique().tolist())
    print(f"Playoff games: {len(po_df)}")

# Save both sets
with open("data/raw/regular_season_ids.pkl", "wb") as f:
    pickle.dump(list(set(regular_ids)), f)

with open("data/raw/playoff_ids.pkl", "wb") as f:
    pickle.dump(list(set(playoff_ids)), f)

print("\n✅ GAME_IDs saved for both Regular Season and Playoffs")



Season: 1983-84
Regular games: 1886
Playoff games: 158

Season: 1984-85
Regular games: 1886
Playoff games: 136

Season: 1985-86
Regular games: 1886
Playoff games: 136

Season: 1986-87
Regular games: 1886
Playoff games: 142

Season: 1987-88
Regular games: 1886
Playoff games: 160

Season: 1988-89
Regular games: 2050
Playoff games: 124

Season: 1989-90
Regular games: 2214
Playoff games: 144

Season: 1990-91
Regular games: 2214
Playoff games: 136

Season: 1991-92
Regular games: 2214
Playoff games: 146

Season: 1992-93
Regular games: 2214
Playoff games: 152

Season: 1993-94
Regular games: 2214
Playoff games: 154

Season: 1994-95
Regular games: 2214
Playoff games: 146

Season: 1995-96
Regular games: 2378
Playoff games: 136

Season: 1996-97
Regular games: 2378
Playoff games: 144

Season: 1997-98
Regular games: 2378
Playoff games: 142

Season: 1998-99
Regular games: 1450
Playoff games: 132

Season: 1999-00
Regular games: 2378
Playoff games: 150

Season: 2000-01
Regular games: 2378
Playoff gam

In [1]:
import pickle
import time
import os
from nba_api.stats.endpoints import boxscoretraditionalv2
import pandas as pd
import glob

# === Load PLAYOFF GAME_IDs ===
with open('data/raw/playoff_ids.pkl', 'rb') as f:
    all_game_ids = pickle.load(f)

# === Check existing batches ===
batch_files = glob.glob('data/raw/boxscores_playoff_batch_*.pkl')
batch_files.sort()

games_done = len(batch_files) * 200
remaining_game_ids = all_game_ids[games_done:]

print(f"✅ Found {len(batch_files)} batches already saved.")
print(f"🔁 Resuming from game {games_done + 1} of {len(all_game_ids)}")
print(f"🗂️  Remaining GAME_IDs to process: {len(remaining_game_ids)}")

# === Parameters ===
batch_size = 200
batch_data = []
failed_ids = []

# === Main Loop ===
for i, game_id in enumerate(remaining_game_ids, start=games_done + 1):
    try:
        print(f"📥 Pulling game {i}: {game_id}")
        time.sleep(1)

        boxscore = boxscoretraditionalv2.BoxScoreTraditionalV2(game_id=game_id)
        df = boxscore.get_data_frames()[0]

        # Filter for bench players
        bench_df = df[df['START_POSITION'] == '']
        bench_df['GAME_ID'] = game_id

        batch_data.append(bench_df)

        # Save every batch
        if i % batch_size == 0:
            batch_num = i // batch_size
            out_path = f'data/raw/boxscores_playoff_batch_{batch_num}.pkl'
            pd.concat(batch_data).to_pickle(out_path)
            print(f"✅ Saved batch {batch_num} to {out_path}")
            batch_data = []

    except Exception as e:
        print(f"❌ Failed to pull game {game_id}: {e}")
        failed_ids.append(game_id)

# Save any leftover records
if batch_data:
    batch_num = (games_done + len(remaining_game_ids)) // batch_size + 1
    out_path = f'data/raw/boxscores_playoff_batch_{batch_num}.pkl'
    pd.concat(batch_data).to_pickle(out_path)
    print(f"✅ Saved final (partial) batch {batch_num}")

# Save failed IDs
if failed_ids:
    with open('logs/failed_playoff_ids.txt', 'a') as f:
        for fail_id in failed_ids:
            f.write(f"{fail_id}\n")
    print(f"⚠️  Logged {len(failed_ids)} failed game IDs")


✅ Found 2 batches already saved.
🔁 Resuming from game 401 of 3440
🗂️  Remaining GAME_IDs to process: 3040
📥 Pulling game 401: 0042100233
❌ Failed to pull game 0042100233: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
📥 Pulling game 402: 0042200211
❌ Failed to pull game 0042200211: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
📥 Pulling game 403: 0040100162


KeyboardInterrupt: 